# Pipeline step by step Implementation

In [1]:
import os
from dataclasses import dataclass
from autogen import ConversableAgent, GroupChat, GroupChatManager
from dotenv import load_dotenv

## Define the System Messages

In [2]:
def get_query_enrichment_system_message() -> str:
    return """You are a car expert who helps gather detailed information about DIY car repair queries.
        
        Your task is to have a conversation with human_proxy to gather:
        1. Car brand and model
        2. Year of manufacture
        3. Specific part or system involved
        4. Nature of the problem or maintenance task

        Guidelines:
        - Ask ONE question at a time directly to human_proxy
        - If any information is already provided in the query, don't ask for it again
        - Wait for human_proxy's response before asking the next question
        - Once all information is gathered, provide a summary in this format:
        
        SUMMARY:
        Vehicle: [Year] [Brand] [Model]
        Part/System: [Specific part]
        Task: [Detailed description of the problem/task]
        Additional Notes: [Any relevant details provided by user]

        After providing the SUMMARY, stop and let rag_agent take over."""

In [3]:
def get_response_enhancement_system_message() -> str:
    return """You are an expert at explaining technical car terminology and making DIY instructions accessible.
    
    Your primary responsibilities:
    1. Identify technical terms and jargon in responses
    2. Add clear explanations for complex concepts
    3. Format responses for maximum readability
    4. Maintain technical accuracy while improving clarity

    Enhancement guidelines:
    1. Technical Term Identification:
       - Identify specialized automotive terms
       - Flag industry-specific abbreviations
       - Note complex procedures
    
    2. Explanation Format:
       - Add a "Technical Terms Explained" section at the end
       - Use simple, everyday language in explanations
       - Include relevant analogies when helpful
    
    3. Formatting:
       - Break down complex procedures into steps
       - Use bullet points for lists
       - Highlight important safety warnings
    
    4. Maintain Original Content:
       - Keep original technical information intact
       - Add explanations without removing details
       - Preserve specific part numbers and specifications
    
    Example format:
    [Original technical response]
    
    Technical Terms Explained:
    - Term 1: Simple explanation
    - Term 2: Simple explanation with analogy
    
    Safety Notes:
    [Any relevant safety information]."""

In [10]:
def get_rag_agent_system_message() -> str:
    return """You handle knowledge base queries for car repair information.
        When you see a SUMMARY from query_enrich_agent:
        1. Extract the relevant information from the SUMMARY
        2. Use the query_knowledge_base function to get repair instructions
        3. Share the retrieved instructions with response_enhancement_agent
        
        Do NOT make up information or rely on general knowledge."""

    # TODO: add prompts that if it doesn't know, let enrich agent to ask more information, change manager system message too

In [11]:
def get_manager_system_message() -> str:
    return """You are managing a car repair help conversation. Follow this EXACT sequence:
        
        CONVERSATION FLOW:
        1. START: Let query_enrich_agent begin with a question to human_proxy
        2. INFORMATION GATHERING:
        - Let query_enrich_agent and human_proxy interact
        - query_enrich_agent should ask ONE question at a time
        - Continue until query_enrich_agent provides a SUMMARY
        3. RAG QUERY:
        - When a SUMMARY appears, direct rag_agent to process it
        - Ensure rag_agent uses query_knowledge_base function
        4. ENHANCEMENT:
        - When rag_agent provides instructions, direct response_enhancement_agent
        - Wait for response with both REPAIR INSTRUCTIONS and TECHNICAL TERMS EXPLAINED
        
        SPEAKER CONTROL:
        - query_enrich_agent starts with a friendly greeting and first question
        - Manage turn-taking between agents based on their responses
        - Ensure each agent completes their task before moving to the next phase
        
        TERMINATION CONDITIONS:
        - End when response is returned from the response_enhancement_agent immediately
        
        Your role is to direct traffic between agents and ensure the conversation follows this sequence."""

## Set up RAG

In [12]:
from agents.agentic_rag_2 import MultiDocumentRAG, root_dir
    
# Initialize RAG system
UPLOAD_DIR = os.path.join(root_dir, "backend/uploads")
PERSIST_DIR = os.path.join(root_dir, "backend/storage")

rag = MultiDocumentRAG(UPLOAD_DIR, PERSIST_DIR)

rag.setup_indicies() # loads existing indicies
rag.create_indices() # creates indices for files that have not yet been indexed
rag.setup_agent()

Loaded index for Tesla_Model_3_Brakes.txt
Loaded index for Tesla_Model_3_Seats.txt
No new files to create indices for
Name: Seat_Cushion_Lower_2nd_Row_Removal_and_Replacement
Description: The document provides detailed instructions for removing and replacing various components in a Tesla Model 3, including the 2nd-row lower seat cushion, brake components, driver footwell duct, knee airbag cover, footwell cover, 12V power connection, and rear underhood apron.
Name: Seat_Back_Rest_Replacement_Procedure_for_Tesla_Model_3
Description: This document provides detailed instructions for removing and replacing the front passenger seat back rest in a Tesla Model 3. It includes steps for removing various components such as the lumbar support, recliner motor, and headrest post guides, as well as guidelines for installing these components back onto the seat assembly. The document also emphasizes safety precautions, such as wearing cut-resistant gloves and proper personal protective equipment.


## Set up function calls

### knowledge base query

the function call will be determined by rag_agent using the summary from entry point

In [13]:
def query_knowledge_base(query: str) -> str:
    """Function to query the RAG system"""
    try:
        response = rag.query(query).response
        return response
    except Exception as e:
        return f"Error querying knowledge base: {str(e)}"

### post query enhancement
`TODO` - the function call will replace the effort of prompt engineering in `response_enhancement_agent`

## Set up Group Chat

In [14]:
def setup_groupchat():
    load_dotenv()
    llm_config = {"config_list": [{"model": 'gpt-4', "api_key": os.environ["OPENAI_API_KEY"]}]}
    
    # Human proxy - for user interaction
    human_proxy = ConversableAgent(
        "human_proxy",
        llm_config=False,
        human_input_mode="ALWAYS"
    )
    
    # Query enrichment agent - focuses on gathering information
    query_enrich_agent = ConversableAgent(
        "query_enrich_agent",
        system_message=get_query_enrichment_system_message(),
        llm_config=llm_config
    )
    
    # RAG agent - handles knowledge base queries
    rag_agent = ConversableAgent(
        "rag_agent",
        system_message=get_rag_agent_system_message(),
        llm_config=llm_config
    )
    
    # Response enhancement agent - explains technical terms
    response_enhancement_agent = ConversableAgent(
        "response_enhancement_agent",
        system_message=get_response_enhancement_system_message(),
        llm_config=llm_config
    )
    
    # Only register the query_knowledge_base function with RAG agent
    rag_agent.register_for_llm(
        description="Queries the car repair knowledge base using the enriched query"
    )(query_knowledge_base)
    rag_agent.register_for_execution()(query_knowledge_base)

    # Create GroupChat with clear flow
    groupchat = GroupChat(
        agents=[human_proxy, query_enrich_agent, rag_agent, response_enhancement_agent],
        messages=[],
        max_round=20  # Increased to allow for multiple questions from query_enrich_agent
    )

    # Manager focuses purely on orchestration
    manager = GroupChatManager(
        groupchat=groupchat,
        llm_config=llm_config,
        system_message=get_manager_system_message()
    )

    return manager, groupchat


def run_car_diy_chat():
    manager, groupchat = setup_groupchat()
    # Start the chat
    manager.initiate_chat(
        manager,
        message="Hello! I am here to help you with your DIY needs"
    )

    # Return the final enhanced response
    chat_history = groupchat.messages
    return chat_history

In [15]:
chat_history = run_car_diy_chat()

chat_manager (to chat_manager):

Hello! I am here to help you with your DIY needs

--------------------------------------------------------------------------------

Next speaker: human_proxy

human_proxy (to chat_manager):

I want to change my car seat

--------------------------------------------------------------------------------

Next speaker: query_enrich_agent


>>>>>>>> USING AUTO REPLY...
query_enrich_agent (to chat_manager):

Sure, I can help with that. Can you tell me the brand and model of your car, please?

--------------------------------------------------------------------------------

Next speaker: human_proxy

human_proxy (to chat_manager):

tesla model 3

--------------------------------------------------------------------------------

Next speaker: query_enrich_agent


>>>>>>>> USING AUTO REPLY...
query_enrich_agent (to chat_manager):

Great! Now, may I know the year of manufacture of your Tesla Model 3?

---------------------------------------------------------------

In [16]:
for message in chat_history:
    print("-------------")
    print(f"{message['name']}: \n{message['content']}")

-------------
chat_manager: 
Hello! I am here to help you with your DIY needs
-------------
human_proxy: 
I want to change my car seat
-------------
query_enrich_agent: 
Sure, I can help with that. Can you tell me the brand and model of your car, please?
-------------
human_proxy: 
tesla model 3
-------------
query_enrich_agent: 
Great! Now, may I know the year of manufacture of your Tesla Model 3?
-------------
human_proxy: 
2023
-------------
query_enrich_agent: 
Thanks for the information. 
Are you looking to replace the entire seat, or just a part of it (like the cover or the cushion)? What seems to be the reason for the change?
-------------
human_proxy: 
my front passenger seat is too dirty, I have to replace it with a new one before tomorrow's trip
-------------
query_enrich_agent: 
SUMMARY:
Vehicle: 2023 Tesla Model 3
Part/System: Front passenger seat
Task: Need to replace the front passenger seat due to dirt
Additional Notes: The task needs to be completed before a trip tomorr

In [17]:
print(len(chat_history))

14


In [18]:
rag.shutdown()